<img src = "https://res.cloudinary.com/cityunscripted/image/upload/s--7t8HSmQ2--/c_fill,f_auto,g_auto:subject,h_270,q_auto,w_375/v1561714908/production/experiences/amsterdam/eat-and-drink-like-a-local-in-amsterdam/ubu2xkch5ihstkm0ruur.jpg" width = 400> </a>

<h1 align=center><font size = 5.5>Capstone Project: Cuisine Journey in Amsterdam with Kristen Phan</font></h1>
<h1 align=center><font size = 4.5>Applied Data Science Capstone Course by IBM/Coursera</font></h1>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

1. [Introduction](#introduction)<br>
2. [Data](#data)<br>
3. [Methodology & Analysis](#methodology) <br>
4. [Results & Discussion](#results) <br>
5. [Conclusion](#conclusion) <br>
6. [About the Author: Kristen Phan](#about) <br>    
</div>
<hr>

## 1. Introduction <a name="introduction"></a>

As a wise man once said, “Food is a central activity of mankind and one of the single most significant trademarks of a culture,”  In this notebook, I will be walking with you along the streets of Amsterdam and cherishing its culture through a new lens that sees through the power of Data Science, stomachs and hearts (*puns intended*).

This notebook is for the Applied Data Science Capstone course. The prompt is that course participants define a problem or an idea of their choice, where they would need to leverage the Foursquare location data via Foursquare API to solve or execute.

Course URL: https://www.coursera.org/learn/applied-data-science-capstone/

## 2. Data <a name="data"></a>

Foursquare is a global location technology company powering location needs for various businesses including Uber and Twitter.
<br>
<br>
In this project, we will obtain information about restaurants in Amsterdam through Foursquare API for analysis and visualization purposes. Details on each restaurant include: type of cuisine (e.g. Italian) and locations (i.e. longtitude and latitude), and pricing points.

In [ ]:
# import libraries
import pandas as pd
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries downloaded!')

In [ ]:
# define Foursquare credentials and version
# @hidden_cell
client_id = 'EX5B52BVRXMSY5CUEZG454WQ0GWVEHHPCV0C23RRRYEQ4Q22' # your Foursquare ID
client_secret = '0OB3TNZZ0RADBXUZW2BXZKMS2DVNXO0APMJT051GSEBBGEER' # your Foursquare Secret
version = '20180605' # Foursquare API version

# @hidden_cell
print('Your credentails:')
print('client_id: ' + client_id)
print('client_secret:' + client_secret)

In [ ]:
# Set parameters for Foursquare API call
category = '4d4b7105d754a06374d81259' # Category ID corresponding to food estabilishments was taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories)
limit = 100
near = 'Amsterdam' # looking up restaurants in Amsterdam

# make an API call to Foursquare to retrive 100 restaurants near Amsterdam
import requests
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&near={}&categoryId={}&limit={}'.format(
        client_id, client_secret, version, near, category, limit)

results = requests.get(url).json()["response"]['groups'][0]['items']
print('Number of restaurants retrieved: '+str(len(results)))

In [ ]:
results

In [ ]:
# create a new dataframe to store venue info
venues_list = []

# return only relevant information for each nearby venue
venues_list.append([( 
            v['venue']['name'],
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = [
                  'venue', 
                  'id',
                  'latitude', 
                  'longitude', 
                  'category']
nearby_venues.head()

In [ ]:
# get an array of unique venue category from nearby_venues dataframe
nearby_venues['category'].unique()

In [ ]:
# create a copy of the nearby_venues dataframe
venues_ethnic = nearby_venues.copy()

# based on the API call results, create a list of generic restaurant category that indicates generic food, not ethic food. These restaurants will be removed from the data frame as the intent is to examine the diversity in ethnic cuisine in Amsterdam as a proxy for the city's diversity in culture 
generic_category = ['Pizza Place', 'Sandwich Place', 'Bakery', 'Breakfast Spot', 'Deli / Bodega', 'Restaurant', 'Food Truck', 'Bistro', 'Gastropub', 'Seafood Restaurant', 'Café', 'Comfort Food Restaurant', 'BBQ Joint', 'Food Stand', 'Food Court', 'Burger Joint', 'Steakhouse', 'Vegetarian / Vegan Restaurant', 'Snack Place']

x = 0 # intialize counter x

# loop through the dataframe and filter out generic restaurants
for i in range(venues_ethnic.shape[0]):
    if venues_ethnic.loc[i,'category'] in generic_category:
        venues_ethnic = venues_ethnic.drop([i])
        x = x + 1 # increment counter x
    
# reset the dataframe's index and display results
venues_ethnic = venues_ethnic.reset_index(drop=True)
print('Number of generic restaurants removed from the list: ', x)
print('Number of ethic restaurants remaining on the list: ', venues_ethnic.shape[0])

# the result only contains restaurants which serve ethinic food
venues_ethnic['category'].unique()

In [ ]:
# display basic info about the final version of venues_ethic dataframe containing restaurants serving ethnic food
venues_ethnic.info()

In [ ]:
venues_ethnic.head()

## 3. Methodology & Analysis <a name="methodology"></a>

After collecting data in the previous step, we will execute the following steps to process and analyze the collected data.

In [ ]:
# store the latitude and longitude of Amsterdam in variables
ams_latitude = 52.370216
ams_longitude = 4.895168


# create map and display it
ams_map = folium.Map(location=[ams_latitude, ams_longitude], zoom_start=12)


# instantiate a feature group for the restaurants in the dataframe
restaurants = folium.map.FeatureGroup()


# loop through the restaurants and add each to the restaurants feature group
for lat, lng, in zip(venues_ethnic.latitude, venues_ethnic.longitude):
    restaurants.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )


# add pop-up text to each marker on the map
latitudes = list(venues_ethnic.latitude)
longitudes = list(venues_ethnic.longitude)
labels = list(venues_ethnic.category)


for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(ams_map)    
    
# add restaurants to map

ams_map.add_child(restaurants)


In [ ]:
# create a new column 'price' to store pricing value for each restaurant
venues_ethnic['price'] = 'NA'
venues_ethnic

In [ ]:
# make an API call and loop through results to filter out the pricing reference for each restaurant
for i in range(venues_ethnic.shape[0]):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venues_ethnic.loc[i, 'id'], client_id, client_secret, version)
    results = requests.get(url).json()
    try:
        venues_ethnic.loc[i,'price'] = results['response']['venue']['price']['message']
    except:
        pass # move onto the next restaurant if the current one does not have a pricing reference on Foursquare

venues_ethnic.head()

In [ ]:
# sort the list of restaurants by category and price. restaurants which have 'NA' for price does not have a pricing reference on Foursquare
sorted_venues_ethinic = venues_ethnic.sort_values(['category','price'])
sorted_venues_ethinic.reset_index(drop=True)

## 4. Results & Discussion <a name="results"></a>

From the above analysis, we come to the following observations:

- Amsterdam has a vibrant, diverse cuisine culture with 18 distinct cuisine options
- As the map indicates, the majority of ethnic restaurants are located in the Centrum neighborhood
- French cuisine seems to be the most well-loved, followed by that of Italian and Japanese
- Restaurants in Amsterdam vary from 'cheap' to 'very expensive', accommodating different customers and dining needs
- If you ever feel like treating yourself to a fancy, French dinner, The Duchess is your destination!

## 5. Conclusion <a name="conclusion"></a>

It was an absolute pleasure to walk with you along the streets of Amsterdam exploring its cuisine culture! Foursquare API coupled with Folium geospatial data visualization package is such a fun way to experience new cities through the power of Data Science. 
<br>
<br>
So, what city would you like to visit next? :)

## 6. About the Author: Kristen Phan <a name="about"></a>

Kristen is passionate about Consulting and Famine Alleviation, both under the lens of Data Science. In her free time, she enjoys reading, backpacking, and spending time with family.
<div style="direction:ltt">
    <a href="https://www.linkedin.com/in/kristenphan/">Connect with Kristen on LinkedIn</a>
</div> 